In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_ = input_[input_['Vehicle_Url'] != ''].reset_index(drop=True)

dict_part = {}
for i in range(len(input_)):
    if input_.loc[i, 'Vehicle_Url'] != '':
        dict_part[input_.loc[i, 'Vehicle_Url']] = {'next': True,
                                                   'crawler': 0,
                                                   'dict_vehicle': {},
                                                   'vehicle_1': '',
                                                   'vehicle_2': ''}

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        while dict_part[input_.loc[a, 'Vehicle_Url']]['next']:
            crawler_status = 'error'
            for b in range(999):
                try:
                    resp = requests.get(input_.loc[a, 'Vehicle_Url'] + '&start=' + str(dict_part[input_.loc[a, 'Vehicle_Url']]['crawler']*50) + '&num=50',
                                        headers=UA.get_User_Agent_Requests(),
                                        proxies=Proxy.get_Proxy_Requests()).text
                    
                    # = = = = = = = = = = = = = = =
                    
                    soup = BeautifulSoup(resp, 'lxml')
                    html = etree.HTML(str(soup))
                    
                    # = = = = = = = = = = = = = = =
                    
                    list_tr = html.xpath('//tr[@class="detail-app-row"]')
                    
                    # = = = = = = = = = = = = = = =

                    for tr in list_tr:
                        make = tr.xpath('./td[2]/text()')[0].strip()
                        model = tr.xpath('./td[3]/text()')[0].strip()
                        engine = tr.xpath('./td[4]/text()')[0].strip()
                        year = int(tr.xpath('./td[1]/text()')[0].strip())

                        if make not in dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle']:
                            dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make] = {model: {engine: [year]}}
                        else:
                            if model not in dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make]:
                                dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make][model] = {engine: [year]}
                            else:
                                if engine not in dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make][model]:
                                    dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make][model][engine] = [year]
                                else:
                                    dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make][model][engine].append(year)

                        dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make][model][engine] = sorted(list(set(dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make][model][engine])))

                    dict_part[input_.loc[a, 'Vehicle_Url']]['crawler'] += 1

                    if len(html.xpath('//a[@aria-label="Next"]')) == 0:
                        dict_part[input_.loc[a, 'Vehicle_Url']]['next'] = False
                    
                    # = = = = = = = = = = = = = = =
                    
                    if not dict_part[input_.loc[a, 'Vehicle_Url']]['next']:
                        list_vehicle = []
                        for make in dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle']:
                            for model in dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make]:
                                list_year = []
                                for engine in dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make][model]:
                                    list_year += dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make][model][engine]
                                list_year = sorted(list(set(list_year)))

                                year_min = min(list_year)
                                year_max = max(list_year)
                                if year_min == year_max:
                                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                                else:
                                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

                        dict_part[input_.loc[a, 'Vehicle_Url']]['vehicle_1'] = '\n'.join(sorted(list_vehicle))
                        
                        # = = = = = = = = = = = = = = =
                        
                        list_vehicle = []
                        for make in dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle']:
                            for model in dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make]:
                                for engine in dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make][model]:
                                    year_min = min(dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make][model][engine])
                                    year_max = max(dict_part[input_.loc[a, 'Vehicle_Url']]['dict_vehicle'][make][model][engine])
                                    if year_min == year_max:
                                        if engine == '':
                                            list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                                        else:
                                            list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + ' ' + engine)
                                    else:
                                        if engine == '':
                                            list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))
                                        else:
                                            list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max) + ' ' + engine)

                        dict_part[input_.loc[a, 'Vehicle_Url']]['vehicle_2'] = '\n'.join(sorted(list_vehicle))

                        # = = = = = = = = = = = = = = =
                        
                        df_temp = pd.DataFrame([{'Vehicle_Url': input_.loc[a, 'Vehicle_Url'],
                                                 'Vehicle_1': dict_part[input_.loc[a, 'Vehicle_Url']]['vehicle_1'],
                                                 'Vehicle_2': dict_part[input_.loc[a, 'Vehicle_Url']]['vehicle_2']}])

                        output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                        
                    # = = = = = = = = = = = = = = =
                    
                    crawler_status = 'ok'
                    
                    break
                except:
                    time.sleep(0.3)
                    continue
                    
            print('[' + crawler_status + '] - ' + input_.loc[a, 'Vehicle_Url'] + ' - ' + str(dict_part[input_.loc[a, 'Vehicle_Url']]['crawler']) +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
                    
            if crawler_status == 'error':
                break

        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Vehicle_Url': input_.loc[a, 'Vehicle_Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
#         print('[' + crawler_status + '] - ' + input_.loc[a, 'Vehicle_Url'] + ' - ' + str(dict_part[input_.loc[a, 'Vehicle_Url']]['crawler']) +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(12):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Vehicle_Url'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./vehicle-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Vehicle_Url'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./vehicle_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：2051

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=102050&PartType=Master%20Cylinder - 1
[尝试次数：1] - [剩余数量：2039] - [当前时间：01:00:06]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=102521&PartType=Master%20Cylinder - 1
[尝试次数：1] - [剩余数量：2038] - [当前时间：01:00:06]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=100162&PartType=Master%20Cylinder - 1
[尝试次数：1] - [剩余数量：2037] - [当前时间：01:00:06]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=103362&PartType=Master%20Cylinder - 1
[尝试次数：1] - [剩余数量：2036] - [当前时间：01:00:06]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=100151&PartType=Master%20Cylinder - 1
[尝试次数：1] - [剩余数量：2035] - [当前时间：01:00:06]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=100358&PartType=Master%20Cylinder - 1
[尝试次数：1] - [剩余数量：2034] - [当前时间：01:00:06]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=102562&PartType=Master%20Cylinder - 1
[尝试

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=11155&PartType=Clutch%20Combination%20Master%20and%20Slave%20Cylinder%20Assy - 1
[尝试次数：2] - [剩余数量：1984] - [当前时间：01:00:17]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=11158&PartType=Clutch%20Master%20Cylinder - 1
[尝试次数：2] - [剩余数量：1983] - [当前时间：01:00:17]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=11159&PartType=Clutch%20Master%20Cylinder - 1
[尝试次数：1] - [剩余数量：1982] - [当前时间：01:00:17]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=11161&PartType=Clutch%20Master%20Cylinder - 1
[尝试次数：2] - [剩余数量：1981] - [当前时间：01:00:18]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=11160&PartType=Clutch%20Master%20Cylinder - 1
[尝试次数：4] - [剩余数量：1980] - [当前时间：01:00:18]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=11157&PartType=Clutch%20Master%20Cylinder - 1
[尝试次数：5] - [剩余数量：1979] - [当前时间：01:00:18]

[ok] - https://www.dormanproducts.c

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=12936&PartType=Clutch%20Master%20Cylinder - 1
[尝试次数：3] - [剩余数量：1937] - [当前时间：01:00:40]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=129385&PartType=Master%20Cylinder - 1
[尝试次数：2] - [剩余数量：1936] - [当前时间：01:00:40]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=12925&PartType=Clutch%20Master%20Cylinder - 1
[尝试次数：11] - [剩余数量：1935] - [当前时间：01:00:40]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=12929&PartType=Clutch%20Master%20Cylinder - 1
[尝试次数：2] - [剩余数量：1934] - [当前时间：01:00:41]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=12934&PartType=Clutch%20Master%20Cylinder - 1
[尝试次数：2] - [剩余数量：1933] - [当前时间：01:00:41]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=129392&PartType=Master%20Cylinder - 1
[尝试次数：2] - [剩余数量：1932] - [当前时间：01:00:42]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=129395&PartType=Ma

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=13248&PartType=Master%20Cylinder - 1
[尝试次数：1] - [剩余数量：1890] - [当前时间：01:01:02]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=13250&PartType=Master%20Cylinder - 1
[尝试次数：2] - [剩余数量：1889] - [当前时间：01:01:03]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=13237&PartType=Master%20Cylinder - 1
[尝试次数：5] - [剩余数量：1888] - [当前时间：01:01:04]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=13255&PartType=Master%20Cylinder - 1
[尝试次数：3] - [剩余数量：1888] - [当前时间：01:01:04]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=13253&PartType=Master%20Cylinder - 1
[尝试次数：2] - [剩余数量：1887] - [当前时间：01:01:04]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=13258&PartType=Master%20Cylinder - 1
[尝试次数：1] - [剩余数量：1886] - [当前时间：01:01:04]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=13238&PartType=Master%20Cylinder - 3
[尝试次数：1] - [剩余数量：188

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=13272&PartType=Master%20Cylinder - 1
[尝试次数：38] - [剩余数量：1843] - [当前时间：01:01:26]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=13550&PartType=Master%20Cylinder - 1
[尝试次数：3] - [剩余数量：1842] - [当前时间：01:01:27]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=13554&PartType=Master%20Cylinder - 1
[尝试次数：2] - [剩余数量：1841] - [当前时间：01:01:27]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=13552&PartType=Master%20Cylinder - 1
[尝试次数：7] - [剩余数量：1840] - [当前时间：01:01:28]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=13555&PartType=Master%20Cylinder - 1
[尝试次数：4] - [剩余数量：1839] - [当前时间：01:01:28]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=13558&PartType=Master%20Cylinder - 1
[尝试次数：3] - [剩余数量：1838] - [当前时间：01:01:29]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=13560&PartType=Master%20Cylinder - 1
[尝试次数：1] - [剩余数量：18

...

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=84720&PartType=Master%20Cylinder - 59
[尝试次数：1] - [剩余数量：0] - [当前时间：01:16:56]

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=84720&PartType=Master%20Cylinder - 60
[尝试次数：1] - [剩余数量：0] - [当前时间：01:16:58]



Traceback (most recent call last):
  File "D:\venv\Lib\site-packages\gevent\_ffi\loop.py", line 270, in python_check_callback
    def python_check_callback(self, watcher_ptr): # pylint:disable=unused-argument
    
KeyboardInterrupt
2024-04-10T17:16:58Z


KeyboardInterrupt: 

[ok] - https://www.dormanproducts.com/itemdetailapp.aspx?ProductID=84720&PartType=Master%20Cylinder - 61
[尝试次数：1] - [剩余数量：0] - [当前时间：01:16:59]



D:\venv\Lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
